In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fruit-classification10-class/MY_data/predict/0124.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/023.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/365.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/659.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/img_241.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/22.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/9.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/011.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/0.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/img_171.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/img_31.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/img_361.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/img_41.jpeg
/kaggle/input/fruit-classification10-class/MY_data/predict/img_21.jpeg
/kaggle/input/fruit-classificat

In [2]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np

# Define image transformations to normalize images to the [0, 1] range
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to 64x64 pixels
    transforms.ToTensor(),  # Convert image to PyTorch tensor and normalize to [0, 1] range automatically
])

# Load the training dataset
train_data_dir = '/kaggle/input/fruit-classification10-class/MY_data/train'
train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)

# Load the test dataset
test_data_dir = '/kaggle/input/fruit-classification10-class/MY_data/test'
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)

# Create DataLoaders for the training and test datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the function to apply Poisson encoding
def poisson_encode(images, time_window=1.0):
    rate = images * time_window  # Scale by the time window to define Poisson rate
    spikes = torch.poisson(rate)
    return spikes

# Encode and save the entire dataset
def encode_and_save_dataset(loader, save_path):
    encoded_data = []
    for batch_idx, (images, labels) in enumerate(loader):
        encoded_images = poisson_encode(images, time_window=1.0)
        encoded_data.append((encoded_images, labels))

    # Save encoded tensors as a single file (can be separated if needed)
    torch.save(encoded_data, save_path)
    print(f"Encoded dataset saved to {save_path}")

# Encode and save the training and test datasets
encode_and_save_dataset(train_loader, "encoded_train_data.pt")
encode_and_save_dataset(test_loader, "encoded_test_data.pt")

Encoded dataset saved to encoded_train_data.pt
Encoded dataset saved to encoded_test_data.pt


In [3]:
import torch

# Load the previously saved encoded datasets
train_data_path = "encoded_train_data.pt"
test_data_path = "encoded_test_data.pt"

encoded_train_data = torch.load(train_data_path)
encoded_test_data = torch.load(test_data_path)

# Combine the datasets
combined_encoded_data = []

# Append training data
for encoded_images, labels in encoded_train_data:
    combined_encoded_data.append((encoded_images, labels))

# Append test data
for encoded_images, labels in encoded_test_data:
    combined_encoded_data.append((encoded_images, labels))

# Save the combined dataset
combined_data_path = "combined_encoded_data.pt"
torch.save(combined_encoded_data, combined_data_path)
print(f"Combined encoded dataset saved to {combined_data_path}")

/tmp/ipykernel_30/1707506036.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoded_train_data = torch.load(train_data_path)
/tmp/ipykernel_30/1707506036.py:8: FutureWa

Combined encoded dataset saved to combined_encoded_data.pt


In [4]:
from torchvision import datasets
import torchvision.transforms as transforms
from collections import Counter

# Define the path to the dataset directory (adjust as necessary)
data_dir = '/kaggle/input/fruit-classification10-class/MY_data/train'

# Define the transformations (e.g., normalization)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Get class-to-index mapping and count the number of images in each class
class_to_idx = dataset.class_to_idx
class_counts = Counter([label for _, label in dataset])

# Print the number of images in each class
for class_name, class_index in class_to_idx.items():
    print(f"Class '{class_name}': {class_counts[class_index]} images")

Class 'Apple': 230 images
Class 'Banana': 230 images
Class 'avocado': 230 images
Class 'cherry': 230 images
Class 'kiwi': 230 images
Class 'mango': 231 images
Class 'orange': 230 images
Class 'pinenapple': 230 images
Class 'strawberries': 230 images
Class 'watermelon': 230 images


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

class PolicyNetwork(nn.Module):
    def __init__(self, num_classes=10):
        super(PolicyNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 256)  # Assuming images are resized to 64x64
        self.fc2 = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

In [6]:
def calculate_reward(probabilities, predicted_class, true_class):
    if predicted_class == true_class:
        return 1.0  # Reward for correct classification
    else:
        confidence = probabilities[0, predicted_class].item()  # Get the confidence in the wrong class
        punishment = -confidence  # The more confident the wrong prediction, the harsher the punishment
        return punishment

In [7]:
def train_policy_gradient(policy_network, dataloader, num_classes=10, num_epochs=10, learning_rate=0.001):
    optimizer = optim.Adam(policy_network.parameters(), lr=learning_rate)
    policy_network.train()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for images, labels in dataloader:
            # Forward pass
            probabilities = policy_network(images)
            predicted_classes = torch.argmax(probabilities, dim=1)

            # Calculate rewards
            rewards = torch.tensor([calculate_reward(probabilities[i].unsqueeze(0), predicted_classes[i], labels[i]) 
                                    for i in range(len(labels))])

            # Calculate the log probability of the predicted classes
            log_probs = torch.log(probabilities[range(len(labels)), predicted_classes])

            # Calculate the policy loss (negative log-likelihood weighted by the rewards)
            policy_loss = -torch.mean(log_probs * rewards)

            # Backpropagation and optimization
            optimizer.zero_grad()
            policy_loss.backward()
            optimizer.step()

            epoch_loss += policy_loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Instantiate the policy network and DataLoader (assuming train_loader is defined)
policy_network = PolicyNetwork(num_classes=10)
train_policy_gradient(policy_network, train_loader, num_classes=10, num_epochs=10, learning_rate=0.001)

Epoch 1/10, Loss: -0.5934
Epoch 2/10, Loss: -0.4400
Epoch 3/10, Loss: -0.1427
Epoch 4/10, Loss: 1.9031
Epoch 5/10, Loss: 4.2430
Epoch 6/10, Loss: 0.4997
Epoch 7/10, Loss: -0.6807
Epoch 8/10, Loss: 0.5461
Epoch 9/10, Loss: 0.4575
Epoch 10/10, Loss: 0.2893


In [15]:
def calculate_accuracy(predictions, labels):
    """Calculate the accuracy of predictions against the true labels."""
    correct = (predictions == labels).sum().item()
    total = labels.size(0)
    accuracy = (correct / total) * 100
    return accuracy

def evaluate_policy_network(policy_network, dataloader):
    """Evaluate the policy network on the given dataloader (test set)."""
    policy_network.eval()  # Set to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            probabilities = policy_network(images)
            predicted_classes = torch.argmax(probabilities, dim=1)
            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)

    return (correct / total) * 100

def train_policy_gradient(policy_network, train_loader, test_loader=None, num_classes=10, num_epochs=10, learning_rate=0.001):
    optimizer = optim.Adam(policy_network.parameters(), lr=learning_rate)
    policy_network.train()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        total_correct_train = 0
        total_samples_train = 0

        # Training loop
        for images, labels in train_loader:
            # Forward pass
            probabilities = policy_network(images)
            predicted_classes = torch.argmax(probabilities, dim=1)

            # Calculate rewards
            rewards = torch.tensor([calculate_reward(probabilities[i].unsqueeze(0), predicted_classes[i], labels[i]) 
                                    for i in range(len(labels))])

            # Calculate the log probability of the predicted classes
            log_probs = torch.log(probabilities[range(len(labels)), predicted_classes])

            # Calculate the policy loss (negative log-likelihood weighted by the rewards)
            policy_loss = -torch.mean(log_probs * rewards)

            # Backpropagation and optimization
            optimizer.zero_grad()
            policy_loss.backward()
            optimizer.step()

            epoch_loss += policy_loss.item()

            # Calculate training accuracy for the batch
            total_correct_train += (predicted_classes == labels).sum().item()
            total_samples_train += labels.size(0)

        # Calculate training accuracy for the epoch
        training_accuracy = (total_correct_train / total_samples_train) * 100

        # Evaluate on the test set if provided
        if test_loader is not None:
            testing_accuracy = evaluate_policy_network(policy_network, test_loader)
        else:
            testing_accuracy = training_accuracy  # If no test set is provided, use training accuracy

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Training Accuracy: {training_accuracy:.2f}%, Testing Accuracy: {testing_accuracy:.2f}%")

# Instantiate the policy network and DataLoader (assuming train_loader and test_loader are defined)
policy_network = PolicyNetwork(num_classes=10)
train_policy_gradient(policy_network, train_loader, test_loader=test_loader, num_classes=10, num_epochs=20, learning_rate=0.001)

Epoch 1/20, Loss: -1.5451, Training Accuracy: 11.73%, Testing Accuracy: 12.78%
Epoch 2/20, Loss: 0.2413, Training Accuracy: 17.73%, Testing Accuracy: 16.59%
Epoch 3/20, Loss: 0.4673, Training Accuracy: 20.90%, Testing Accuracy: 21.27%
Epoch 4/20, Loss: 0.7379, Training Accuracy: 28.94%, Testing Accuracy: 28.39%
Epoch 5/20, Loss: -0.8182, Training Accuracy: 37.16%, Testing Accuracy: 27.71%
Epoch 6/20, Loss: -0.2394, Training Accuracy: 38.20%, Testing Accuracy: 32.20%
Epoch 7/20, Loss: -1.1973, Training Accuracy: 42.16%, Testing Accuracy: 35.51%
Epoch 8/20, Loss: -0.4266, Training Accuracy: 44.72%, Testing Accuracy: 31.71%
Epoch 9/20, Loss: -0.8701, Training Accuracy: 47.54%, Testing Accuracy: 35.22%
Epoch 10/20, Loss: 0.2449, Training Accuracy: 51.15%, Testing Accuracy: 38.63%
Epoch 11/20, Loss: 1.4625, Training Accuracy: 58.50%, Testing Accuracy: 40.98%
Epoch 12/20, Loss: 0.4340, Training Accuracy: 62.15%, Testing Accuracy: 40.39%
Epoch 13/20, Loss: 0.5796, Training Accuracy: 65.75%, T

In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

# Define the Residual Block for enhanced gradient flow
class ResidualBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ResidualBlock, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x
        out = self.relu(self.bn(self.fc(x)))
        return out + identity  # Skip connection

# Define the Enhanced Policy Network with Residual Blocks and Dynamic Dropout
class ResidualPolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ResidualPolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.residual1 = ResidualBlock(128, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, output_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = nn.ReLU()(self.bn1(self.fc1(x)))
        x = self.residual1(x)
        x = self.dropout1(x)
        action_probs = self.softmax(self.fc2(x))
        return action_probs

# Normalize data to range [0, 1]
def normalize_data(images):
    return (images - images.min()) / (images.max() - images.min() + 1e-6)

# Advanced data augmentation using additional transformations
data_augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
])

# Function to train the policy network using cross-entropy loss and exponential learning rate decay
def train_with_residuals(train_loader, test_loader, input_dim, num_classes, num_epochs=30, initial_lr=1e-3, early_stopping_patience=5):
    policy_net = ResidualPolicyNetwork(input_dim, num_classes)
    optimizer = optim.SGD(policy_net.parameters(), lr=initial_lr, momentum=0.9, weight_decay=1e-3)

    # Exponential learning rate decay
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)  # Slow exponential decay

    # Label smoothing loss
    criterion = nn.CrossEntropyLoss(label_smoothing=0.05)  # Reduced label smoothing

    best_validation_accuracy = 0.0
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        policy_net.train()
        epoch_loss = 0.0
        total, correct = 0, 0

        for images, labels in train_loader:
            # Apply data augmentation
            images = torch.stack([data_augmentation(image) for image in images])
            images = images.view(images.size(0), -1)  # Flatten the images
            images = normalize_data(images)  # Normalize input data
            optimizer.zero_grad()

            # Forward pass through the policy network
            action_probs = policy_net(images)
            predicted_classes = torch.argmax(action_probs, dim=1)

            # Calculate loss
            loss = criterion(action_probs, labels)

            # Update the policy network
            loss.backward()
            optimizer.step()

            # Accumulate epoch statistics
            epoch_loss += loss.item()
            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)

        # Print training statistics for the epoch
        train_accuracy = (correct / total) * 100
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

        # Validate the policy network and print test accuracy
        validation_accuracy = validate_policy_network(test_loader, policy_net)

        # Learning rate scheduler step
        scheduler.step()

        # Early stopping logic
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            early_stopping_counter = 0  # Reset early stopping counter
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping due to no improvement in validation accuracy.")
            break

# Function to validate the policy network
def validate_policy_network(loader, model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1)
            images = normalize_data(images)  # Normalize input data
            action_probs = model(images)
            predicted_classes = torch.argmax(action_probs, dim=1)
            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)
    validation_accuracy = (correct / total) * 100
    print(f"Validation Accuracy: {validation_accuracy:.2f}%")
    return validation_accuracy

# Assuming 'encoded_train_loader' and 'encoded_test_loader' contain the Poisson encoded training and test data
input_dim = 64 * 64 * 3  # Input dimension based on image size (64x64 RGB images)
num_classes = 10  # Number of classes in the dataset

# Train the policy gradient model with residuals and exponential learning rate decay
train_with_residuals(encoded_train_loader, encoded_test_loader, input_dim, num_classes, num_epochs=30, initial_lr=1e-3, early_stopping_patience=5)

Epoch 1/30, Loss: 165.5002, Train Accuracy: 11.99%
Validation Accuracy: 12.88%
Epoch 2/30, Loss: 163.7529, Train Accuracy: 18.04%
Validation Accuracy: 17.37%
Epoch 3/30, Loss: 162.2232, Train Accuracy: 22.12%
Validation Accuracy: 18.54%
Epoch 4/30, Loss: 160.8865, Train Accuracy: 24.64%
Validation Accuracy: 18.63%
Epoch 5/30, Loss: 159.8436, Train Accuracy: 26.68%
Validation Accuracy: 20.88%
Epoch 6/30, Loss: 159.3880, Train Accuracy: 26.38%
Validation Accuracy: 23.02%
Epoch 7/30, Loss: 158.6503, Train Accuracy: 28.86%
Validation Accuracy: 22.54%
Epoch 8/30, Loss: 158.5538, Train Accuracy: 27.03%
Validation Accuracy: 23.41%
Epoch 9/30, Loss: 157.7525, Train Accuracy: 29.42%
Validation Accuracy: 23.51%
Epoch 10/30, Loss: 157.7915, Train Accuracy: 29.12%
Validation Accuracy: 23.51%
Epoch 11/30, Loss: 157.4236, Train Accuracy: 30.60%
Validation Accuracy: 22.63%
Epoch 12/30, Loss: 157.0007, Train Accuracy: 30.12%
Validation Accuracy: 23.41%
Epoch 13/30, Loss: 157.0423, Train Accuracy: 29.9

In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# Define an Extended Residual Block for better generalization
class ExtendedResidualBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ExtendedResidualBlock, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)
        self.bn1 = nn.BatchNorm1d(output_dim)
        self.relu = nn.LeakyReLU(negative_slope=0.1)
        self.fc2 = nn.Linear(output_dim, output_dim)
        self.bn2 = nn.BatchNorm1d(output_dim)

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.fc1(x)))
        out = self.bn2(self.fc2(out))
        return self.relu(out + identity)  # Skip connection with activation

# Define the Updated Policy Network with More Regularization
class RefinedPolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RefinedPolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.residual1 = ExtendedResidualBlock(256, 256)
        self.residual2 = ExtendedResidualBlock(256, 256)
        self.dropout1 = nn.Dropout(0.5)  # Increased dropout
        self.fc2 = nn.Linear(256, output_dim)
        self.softmax = nn.Softmax(dim=-1)

        # Xavier Normal initialization for weights
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)

    def forward(self, x):
        x = nn.LeakyReLU(negative_slope=0.1)(self.bn1(self.fc1(x)))
        x = self.residual1(x)
        x = self.residual2(x)
        x = self.dropout1(x)
        action_probs = self.softmax(self.fc2(x))
        return action_probs

# Normalize data to range [0, 1]
def normalize_data(images):
    return (images - images.min()) / (images.max() - images.min() + 1e-6)

# Enhanced Reward and Punishment Calculations
def calculate_rewards_and_punishments(action_probs, predicted_classes, labels):
    correct_mask = (predicted_classes == labels)
    confidence_correct = action_probs[correct_mask, predicted_classes[correct_mask]]
    rewards = torch.zeros_like(action_probs[:, 0])
    rewards[correct_mask] = 1.0 + (confidence_correct * 1.5)

    incorrect_mask = ~correct_mask
    confidence_incorrect = action_probs[incorrect_mask].max(dim=1)[0]
    punishments = torch.zeros_like(rewards)
    punishments[incorrect_mask] = confidence_incorrect * 2.0

    return rewards, punishments

# Train with Advanced Regularization and Data Augmentation
def train_with_refined_techniques(train_loader, test_loader, input_dim, num_classes, num_epochs=30, initial_lr=1e-4, early_stopping_patience=7):
    policy_net = RefinedPolicyNetwork(input_dim, num_classes)
    optimizer = optim.Adam(policy_net.parameters(), lr=initial_lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    best_validation_accuracy = 0.0
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        policy_net.train()
        epoch_loss = 0.0
        total, correct = 0, 0
        epoch_rewards = 0.0

        for images, labels in train_loader:
            images = images.view(images.size(0), -1)  # Flatten the images
            images = normalize_data(images)
            optimizer.zero_grad()

            # Forward pass through the policy network
            action_probs = policy_net(images)
            predicted_classes = torch.argmax(action_probs, dim=1)

            # Calculate rewards and punishments
            rewards, punishments = calculate_rewards_and_punishments(action_probs, predicted_classes, labels)
            total_reward = rewards.sum()
            total_punishment = punishments.sum()

            # Policy gradient loss
            action_log_probs = torch.log(action_probs + 1e-6)
            selected_action_log_probs = action_log_probs[range(images.size(0)), predicted_classes]
            loss = -torch.mean(selected_action_log_probs * (rewards - punishments))

            # Update the policy network
            loss.backward()
            optimizer.step()

            # Accumulate epoch statistics
            epoch_loss += loss.item()
            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)
            epoch_rewards += total_reward.item()

        train_accuracy = (correct / total) * 100
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Rewards: {epoch_rewards:.2f}, Train Accuracy: {train_accuracy:.2f}%")

        # Validate the policy network and print test accuracy
        validation_accuracy = validate_policy_network(test_loader, policy_net)

        # Dynamic learning rate scheduler step based on validation loss
        scheduler.step(epoch_loss)

        # Early stopping logic
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping due to no improvement in validation accuracy.")
            break

# Function to validate the policy network
def validate_policy_network(loader, model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1)
            images = normalize_data(images)  # Normalize input data
            action_probs = model(images)
            predicted_classes = torch.argmax(action_probs, dim=1)
            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)
    validation_accuracy = (correct / total) * 100
    print(f"Validation Accuracy: {validation_accuracy:.2f}%")
    return validation_accuracy

# Assuming 'encoded_train_loader' and 'encoded_test_loader' contain the Poisson encoded training and test data
input_dim = 64 * 64 * 3
num_classes = 10

# Train the policy gradient model with improved regularization and reward mechanisms
train_with_refined_techniques(encoded_train_loader, encoded_test_loader, input_dim, num_classes, num_epochs=30, initial_lr=1e-4, early_stopping_patience=7)

Epoch 1/30, Loss: -28.0488, Rewards: 464.72, Train Accuracy: 10.52%
Validation Accuracy: 12.10%
Epoch 2/30, Loss: -22.9257, Rewards: 813.46, Train Accuracy: 17.38%
Validation Accuracy: 12.68%
Epoch 3/30, Loss: -19.9737, Rewards: 1147.32, Train Accuracy: 23.42%
Validation Accuracy: 14.83%
Epoch 4/30, Loss: -14.5555, Rewards: 1566.73, Train Accuracy: 31.20%
Validation Accuracy: 15.12%
Epoch 5/30, Loss: -12.4974, Rewards: 1906.55, Train Accuracy: 37.16%
Validation Accuracy: 16.98%
Epoch 6/30, Loss: -10.2442, Rewards: 2313.04, Train Accuracy: 43.98%
Validation Accuracy: 18.54%
Epoch 7/30, Loss: -9.2814, Rewards: 2584.12, Train Accuracy: 48.33%
Validation Accuracy: 19.32%
Epoch 8/30, Loss: -6.5714, Rewards: 2868.24, Train Accuracy: 53.59%
Validation Accuracy: 20.88%
Epoch 9/30, Loss: -4.1328, Rewards: 3145.34, Train Accuracy: 58.45%
Validation Accuracy: 20.88%
Epoch 10/30, Loss: -4.4058, Rewards: 3351.01, Train Accuracy: 61.45%
Validation Accuracy: 21.37%
Epoch 11/30, Loss: -4.7394, Rewards

In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# Define an Extended Residual Block for better generalization
class ExtendedResidualBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ExtendedResidualBlock, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)
        self.bn1 = nn.BatchNorm1d(output_dim)
        self.relu = nn.LeakyReLU(negative_slope=0.1)
        self.fc2 = nn.Linear(output_dim, output_dim)
        self.bn2 = nn.BatchNorm1d(output_dim)

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.fc1(x)))
        out = self.bn2(self.fc2(out))
        return self.relu(out + identity)  # Skip connection with activation

# Define the Updated Policy Network with More Regularization
# Define the Updated Policy Network with More Regularization and Additional Hidden Layers
class RefinedPolicyNetworkWithMoreHiddenLayers(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RefinedPolicyNetworkWithMoreHiddenLayers, self).__init__()
        
        # Define a dynamic way to add hidden layers
        hidden_layers = []
        current_dim = 6144  # Starting with 6144 as per your suggestion
        
        # Add layers halving the number of neurons until reaching 256
        while current_dim >= 256:
            hidden_layers.append(nn.Linear(input_dim, current_dim))
            hidden_layers.append(nn.BatchNorm1d(current_dim))
            hidden_layers.append(nn.LeakyReLU(negative_slope=0.1))
            input_dim = current_dim
            current_dim //= 2
        
        # Combine all the layers
        self.hidden_layers = nn.Sequential(*hidden_layers)
        
        # Output layer with softmax
        self.output_layer = nn.Linear(input_dim, output_dim)
        self.softmax = nn.Softmax(dim=-1)

        # Initialize weights using Xavier Normal
        for layer in hidden_layers:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
        nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, x):
        x = self.hidden_layers(x)
        action_probs = self.softmax(self.output_layer(x))
        return action_probs

# Train function and other components remain the same


# Normalize data to range [0, 1]
def normalize_data(images):
    return (images - images.min()) / (images.max() - images.min() + 1e-6)

# Enhanced Reward and Punishment Calculations
def calculate_rewards_and_punishments(action_probs, predicted_classes, labels):
    correct_mask = (predicted_classes == labels)
    confidence_correct = action_probs[correct_mask, predicted_classes[correct_mask]]
    rewards = torch.zeros_like(action_probs[:, 0])
    rewards[correct_mask] = 1.0 + (confidence_correct * 1.5)

    incorrect_mask = ~correct_mask
    confidence_incorrect = action_probs[incorrect_mask].max(dim=1)[0]
    punishments = torch.zeros_like(rewards)
    punishments[incorrect_mask] = confidence_incorrect * 2.0

    return rewards, punishments

# Train with Advanced Regularization and Data Augmentation
def train_with_refined_techniques(train_loader, test_loader, input_dim, num_classes, num_epochs=30, initial_lr=1e-4, early_stopping_patience=7):
    policy_net = RefinedPolicyNetwork(input_dim, num_classes)
    optimizer = optim.Adam(policy_net.parameters(), lr=initial_lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    best_validation_accuracy = 0.0
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        policy_net.train()
        epoch_loss = 0.0
        total, correct = 0, 0
        epoch_rewards = 0.0

        for images, labels in train_loader:
            images = images.view(images.size(0), -1)  # Flatten the images
            images = normalize_data(images)
            optimizer.zero_grad()

            # Forward pass through the policy network
            action_probs = policy_net(images)
            predicted_classes = torch.argmax(action_probs, dim=1)

            # Calculate rewards and punishments
            rewards, punishments = calculate_rewards_and_punishments(action_probs, predicted_classes, labels)
            total_reward = rewards.sum()
            total_punishment = punishments.sum()

            # Policy gradient loss
            action_log_probs = torch.log(action_probs + 1e-6)
            selected_action_log_probs = action_log_probs[range(images.size(0)), predicted_classes]
            loss = -torch.mean(selected_action_log_probs * (rewards - punishments))

            # Update the policy network
            loss.backward()
            optimizer.step()

            # Accumulate epoch statistics
            epoch_loss += loss.item()
            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)
            epoch_rewards += total_reward.item()

        train_accuracy = (correct / total) * 100
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Rewards: {epoch_rewards:.2f}, Train Accuracy: {train_accuracy:.2f}%")

        # Validate the policy network and print test accuracy
        validation_accuracy = validate_policy_network(test_loader, policy_net)

        # Dynamic learning rate scheduler step based on validation loss
        scheduler.step(epoch_loss)

        # Early stopping logic
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping due to no improvement in validation accuracy.")
            break

# Function to validate the policy network
def validate_policy_network(loader, model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1)
            images = normalize_data(images)  # Normalize input data
            action_probs = model(images)
            predicted_classes = torch.argmax(action_probs, dim=1)
            correct += (predicted_classes == labels).sum().item()
            total += labels.size(0)
    validation_accuracy = (correct / total) * 100
    print(f"Validation Accuracy: {validation_accuracy:.2f}%")
    return validation_accuracy

# Assuming 'encoded_train_loader' and 'encoded_test_loader' contain the Poisson encoded training and test data
input_dim = 64 * 64 * 3
num_classes = 10

# Train the policy gradient model with improved regularization and reward mechanisms
train_with_refined_techniques(encoded_train_loader, encoded_test_loader, input_dim, num_classes, num_epochs=30, initial_lr=1e-4, early_stopping_patience=7)

Epoch 1/30, Loss: -25.9743, Rewards: 521.66, Train Accuracy: 11.56%
Validation Accuracy: 12.29%
Epoch 2/30, Loss: -22.2606, Rewards: 826.72, Train Accuracy: 17.47%
Validation Accuracy: 13.27%
Epoch 3/30, Loss: -18.3233, Rewards: 1178.13, Train Accuracy: 23.90%
Validation Accuracy: 15.80%
Epoch 4/30, Loss: -15.3968, Rewards: 1526.54, Train Accuracy: 29.99%
Validation Accuracy: 15.32%
Epoch 5/30, Loss: -13.7135, Rewards: 1845.69, Train Accuracy: 35.42%
Validation Accuracy: 17.66%
Epoch 6/30, Loss: -12.4367, Rewards: 2173.46, Train Accuracy: 40.90%
Validation Accuracy: 16.68%
Epoch 7/30, Loss: -9.9374, Rewards: 2442.74, Train Accuracy: 45.76%
Validation Accuracy: 16.29%
Epoch 8/30, Loss: -8.1763, Rewards: 2689.16, Train Accuracy: 50.02%
Validation Accuracy: 17.07%
Epoch 9/30, Loss: -6.9356, Rewards: 2918.59, Train Accuracy: 53.93%
Validation Accuracy: 17.17%
Epoch 10/30, Loss: -5.5938, Rewards: 3151.49, Train Accuracy: 58.02%
Validation Accuracy: 17.66%
Epoch 11/30, Loss: -6.3595, Rewards